In [ ]:

import requests
import os
from dotenv import load_dotenv
from datetime import datetime
import time
load_dotenv() 

swparse_api_key = os.environ["SWPARSE_KEY"]
BASE_URL =  os.environ["BASE_URL"]

files = [
#  '2024SalesPresentationC6501-PPOs.pdf',
 'CMS_AI_Playbook_3_Final.pdf',
#  'My-Agreements-in-4i-Tip-Sheet_508-1.pdf'
]

pdf_filename = files[0]

with open(f"pdf/{pdf_filename}", mode="rb") as f:
 
    files = {
        'file': (pdf_filename, f, 'application/pdf')
    }
    data = {
        "force_ocr": True,
        # "plain_text": True
    }
    
    headers = {
        'Authorization': f'Bearer {swparse_api_key}'
    }
    try:
        start_time = datetime.now()
        print(f"File:{pdf_filename} uploaded")
        response = requests.post(f"{BASE_URL}/api/parsing/upload", files=files, headers=headers, data = data)
        if response.status_code == 201:
            print("File uploaded successfully!")
            res  = response.json()
            print(res)

            job_id = res["id"]
     
        else:
            print(f"Failed to upload the file. Status code: {response.status_code}")
            print(response.text)
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        
    result_type ="markdown"
    if data.get("plain_text"):
        result_type = "text" # plain text only has 'text' result type
    while True:
  
        response = requests.get(f"{BASE_URL}/api/parsing/job/{job_id}/result/{result_type}", headers=headers)
         
        if response.status_code == 200:
            end_time = datetime.now()
            break

        time.sleep(.5)
 
    result = response.json()
    output = result[result_type]
    
    time_taken = end_time - start_time
    minutes, seconds = divmod(time_taken.seconds, 60)
    milliseconds = time_taken.microseconds // 1000
    
    print(f"Time Taken: {minutes} min {seconds} sec {milliseconds} ms")
    
    md_filename = pdf_filename.replace("pdf","txt")
    with open(f"output/{md_filename}", "w") as f:
        f.write(output)
    
    
        

File:CMS_AI_Playbook_3_Final.pdf uploaded
File uploaded successfully!
{'id': 'saq:job:swparse:fba208b4-ce7b-11ef-9f89-0242ac120006', 'status': 'PENDING', 's3_url': 'swparse/2340cd88df33ff3c70188c3cbd2a9476.pdf'}
